# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [8]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)


In [15]:
df.info()
df.describe()
df.isnull().sum()  # Checking missing values
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10910 entries, 0 to 10909
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     10910 non-null  int64  
 1   Customer                       10910 non-null  object 
 2   State                          10279 non-null  object 
 3   Customer Lifetime Value        10910 non-null  float64
 4   Response                       10279 non-null  object 
 5   Coverage                       10910 non-null  object 
 6   Education                      10910 non-null  object 
 7   Effective To Date              10910 non-null  object 
 8   EmploymentStatus               10910 non-null  object 
 9   Gender                         10910 non-null  object 
 10  Income                         10910 non-null  int64  
 11  Location Code                  10910 non-null  object 
 12  Marital Status                 10910 non-null 

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

In [25]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

filtered_df = df[(df['total_claim_amount'] > 1000) & (df['response'] == 'Yes')]
filtered_df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10910 entries, 0 to 10909
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   unnamed:_0                     10910 non-null  int64  
 1   customer                       10910 non-null  object 
 2   state                          10279 non-null  object 
 3   customer_lifetime_value        10910 non-null  float64
 4   response                       10279 non-null  object 
 5   coverage                       10910 non-null  object 
 6   education                      10910 non-null  object 
 7   effective_to_date              10910 non-null  object 
 8   employmentstatus               10910 non-null  object 
 9   gender                         10910 non-null  object 
 10  income                         10910 non-null  int64  
 11  location_code                  10910 non-null  object 
 12  marital_status                 10910 non-null 

,unnamed:_0,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
count,10910.000000,10910.000000,10910.000000,10910.000000,10277.000000,10910.000000,10277.000000,10910.000000,10910.000000
mean,5454.500000,8018.241094,37536.284785,93.196059,15.149071,48.091934,0.384256,2.979193,434.888330
std,3149.590053,6885.081434,30359.195670,34.442532,10.080349,27.940675,0.912457,2.399359,292.180556
min,0.000000,1898.007675,0.000000,61.000000,0.000000,0.000000,0.000000,1.000000,0.099007
25%,2727.250000,4014.453113,0.000000,68.000000,6.000000,24.000000,0.000000,1.000000,271.082527
50%,5454.500000,5771.147235,33813.500000,83.000000,14.000000,48.000000,0.000000,2.000000,382.564630
75%,8181.750000,8992.779137,62250.750000,109.000000,23.000000,71.000000,0.000000,4.000000,547.200000
max,10909.000000,83325.381190,99981.000000,298.000000,35.000000,99.000000,5.000000,9.000000,2893.239678


There is high variability across income, claims, and customer lifetime value, meaning different customer groups have unique behaviors.

Customers who have been with the company longer tend to have higher CLV.

Most customers don’t have open complaints, but a small subset does, which could indicate a need for customer service improvements.

A significant portion of customers is low-income or unemployed, which might impact policy offerings and affordability.

In [21]:
avg_claims = df[df['response'] == 'Yes'].groupby(['policy_type', 'gender'])['total_claim_amount'].mean().reset_index()
print(avg_claims)


      policy_type gender  total_claim_amount
0  Corporate Auto      F          433.738499
1  Corporate Auto      M          408.582459
2   Personal Auto      F          452.965929
3   Personal Auto      M          457.010178
4    Special Auto      F          453.280164
5    Special Auto      M          429.527942


Across all policy types, female customers have slightly higher average claim amounts than male customers.

The largest difference in claim amounts is seen in corporate auto policies, where female customers tend to claim more on average.

In personal auto policies, the claim amounts are nearly identical, showing minimal variation in claim behavior.

Understanding these trends can help the insurance company adjust policy pricing, risk assessment, and marketing strategies based on customer gender and policy type.

In [23]:
state_counts = df.groupby('state')['customer'].count().reset_index()
filtered_states = state_counts[state_counts['customer'] > 500]
print(filtered_states)


        state  customer
0     Arizona      1937
1  California      3552
2      Nevada       993
3      Oregon      2909
4  Washington       888


California has the highest number of customers (3,552), making it the strongest market for the insurance company. This could indicate a larger customer base due to population size or strong brand presence.
The company might want to invest more in marketing and retention strategies in California since it has the largest customer base.
Arizona and Oregon are also high-priority states, where optimizing services and promotions could further expand customer engagement.
Nevada and Washington may require additional strategies to boost customer acquisition, depending on competition and demand.

In [24]:
clv_stats = df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).reset_index()
print(clv_stats)


              education gender          max          min       median
0              Bachelor      F  73225.95652  1904.000852  5640.505303
1              Bachelor      M  67907.27050  1898.007675  5548.031892
2               College      F  61850.18803  1898.683686  5623.611187
3               College      M  61134.68307  1918.119700  6005.847375
4                Doctor      F  44856.11397  2395.570000  5332.462694
5                Doctor      M  32677.34284  2267.604038  5577.669457
6  High School or Below      F  55277.44589  2144.921535  6039.553187
7  High School or Below      M  83325.38119  1940.981221  6286.731006
8                Master      F  51016.06704  2417.777032  5729.855012
9                Master      M  50568.25912  2272.307310  5579.099207


Customers with lower education levels—particularly men—show the highest CLV potential.

Surprisingly, customers with Doctorate degrees tend to have lower median CLV, possibly due to different insurance needs or investment strategies.

The gap between male and female customers is minimal in most education categories, but bachelor's degree holders show significant differences in CLV range.


In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)


In [3]:
print(df.columns)


Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')


In [4]:
df['month'] = pd.to_datetime(df['Effective To Date']).dt.month  # Correct column name
policies_by_state_month = df.groupby(['State', 'month'])['Policy'].count().unstack()
print(policies_by_state_month)


month          1     2
State                 
Arizona     1008   929
California  1918  1634
Nevada       551   442
Oregon      1565  1344
Washington   463   425


/var/folders/ck/t8ktvnl92454zslcr12m5v9w0000gn/T/ipykernel_2728/2143987549.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['month'] = pd.to_datetime(df['Effective To Date']).dt.month  # Correct column name


In [6]:
top_states = df.groupby('State')['Number of Policies'].count().nlargest(3).index  # Find top 3 states
filtered_df = df[df['State'].isin(top_states)]  # Filter for only the top 3 states
policies_top_states = filtered_df.groupby(['State', 'month'])['Number of Policies'].count().unstack()
print(policies_top_states)


month          1     2
State                 
Arizona     1008   929
California  1918  1634
Oregon      1565  1344


In [8]:
import pandas as pd

# Load the dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

# Ensure column names are correct
df.rename(columns=lambda x: x.strip(), inplace=True)

# Filter only relevant columns
df_melted = df.melt(id_vars=['Response'], value_vars=['Sales Channel'], var_name='Marketing Channel', value_name='Channel Name')

# Calculate response rate by marketing channel
response_rate = df_melted[df_melted['Response'] == "Yes"].groupby('Channel Name').size() / df_melted.groupby('Channel Name').size()

# Convert response rate into a DataFrame
response_rate_df = response_rate.reset_index(name='Response Rate')

# Display the final result
print(response_rate_df)


  Channel Name  Response Rate
0        Agent       0.180053
1       Branch       0.107876
2  Call Center       0.103223
3          Web       0.108856
